In [2]:
import duckdb
import os
import tqdm
import pandas as pd

# Store MIMIC III data as Parquet files

In [4]:
#per chatgpt, used for progress bar estimations below
mimic_table_row_counts = { 
    'ADMISSIONS': 58976,
    'CALLOUT': 34499,
    'CAREGIVERS': 7567,
    'CHARTEVENTS': 330712483,
    'CPTEVENTS': 573146,
    'D_CPT': 134,
    'D_ICD_DIAGNOSES': 14710,
    'D_ICD_PROCEDURES': 3898,
    'D_ITEMS': 12487,
    'D_LABITEMS': 753,
    'DATETIMEEVENTS': 4485937,
    'DIAGNOSES_ICD': 651047,
    'DRGCODES': 125557,
    'ICUSTAYS': 61532,
    'INPUTEVENTS_CV': 17527935,
    'INPUTEVENTS_MV': 3618991,
    'LABEVENTS': 27854055,
    'MICROBIOLOGYEVENTS': 631726,
    'NOTEEVENTS': 2083180,
    'OUTPUTEVENTS': 4349218,
    'PATIENTS': 46520,
    'PRESCRIPTIONS': 4157756,
    'PROCEDUREEVENTS_MV': 258066,
    'PROCEDURES_ICD': 240095,
    'SERVICES': 733241,
    'TRANSFERS': 261897
}


In [8]:
# build column list for preprocessing into parquet file
datetime_columns_translation = [
    'ADMITTIME',  # ADMISSIONS
    'DISCHTIME',  # ADMISSIONS
    'DEATHTIME',  # ADMISSIONS
    'EDREGTIME',  # ADMISSIONS
    'EDOUTTIME',  # ADMISSIONS
    'CHARTTIME',  # CHARTEVENTS, DATETIMEEVENTS, LABEVENTS, OUTPUTEVENTS, NOTEEVENTS, MICROBIOLOGYEVENTS
    'STORETIME',  # CHARTEVENTS, DATETIMEEVENTS, OUTPUTEVENTS
    'STARTTIME',  # INPUTEVENTS_MV, PROCEDUREEVENTS_MV
    'ENDTIME',  # INPUTEVENTS_MV, PROCEDUREEVENTS_MV
    'INTIME',  # ICUSTAYS
    'OUTTIME',  # ICUSTAYS
    'DOB',  # PATIENTS 
    'DOD',  # PATIENTS 
    'DOD_HOSP',  # PATIENTS 
    'DOD_SSN',  # PATIENTS 
    'CHARTDATE',  # NOTEEVENTS, MICROBIOLOGYEVENTS
    'STARTDATE',  # PRESCRIPTIONS
    'ENDDATE',  # PRESCRIPTIONS
]

numeric_columns_translation = [
    'VALUENUM',  # Common in multiple tables
    'VALUE',  # CHARTEVENTS, LABEVENTS, DATETIMEEVENTS, etc.
    'AMOUNT',  # INPUTEVENTS_CV, INPUTEVENTS_MV
    'RATE',  # INPUTEVENTS_CV, INPUTEVENTS_MV
    'ORIGINALAMOUNT',  # INPUTEVENTS_CV, INPUTEVENTS_MV
    'ORIGINALRATE',  # INPUTEVENTS_CV, INPUTEVENTS_MV
    'DILUTION_TEXT',  # MICROBIOLOGYEVENTS
    'DILUTION_COMMENTS',  # MICROBIOLOGYEVENTS
    'DOSE_VAL_RX',  # PRESCRIPTIONS
    'FORM_VAL_DISP',  # PRESCRIPTIONS
    'CPT_CD' 
]


In [10]:
# process the extracted files and turn them into parquet files for dask or duckDB
location = "./mimicIII/mimic-iii-clinical-database-1.4"
files_to_process = os.listdir("./mimicIII/mimic-iii-clinical-database-1.4")
#files_to_process = ["/".join([location,file]) for file in files_to_process]

for i, src_file in enumerate(files_to_process[25:]):
    print(f"Processing {src_file} / {1+i} of {len(files_to_process)}")
    if not src_file.endswith(".csv.gz"):
        print(f"\t Skipping since not proper type")
        continue
    if src_file.startswith("CHARTEVENTS"):
        print(f"\t Already loaded CHARTEVENTS")
        continue
    
    base_fn = src_file.replace(".csv.gz", "")

    #print(f"{base_fn} = {mimic_table_row_counts.get(base_fn)}")

    chunksize = 10e6
    reader = pd.read_csv("/".join([location,src_file]), chunksize=chunksize, compression='gzip')  

    
    for i, chunk in enumerate(tqdm.tqdm(reader, total=1+mimic_table_row_counts.get(base_fn)//chunksize, desc="Processing Chunks")):
        # convert numerical values
        num_columns = list(set(numeric_columns_translation).intersection(chunk.columns))
        dt_columns = list(set(datetime_columns_translation).intersection(chunk.columns))
        for c, col in enumerate(num_columns): 
            #print(f"\t\tChanging numeric in {c+1} of {len(num_columns)}")               
            chunk[col] = pd.to_numeric(chunk[col], errors="coerce")
        
        for c, col in enumerate(dt_columns): 
            #print(f"\t\tChanging dt in {c+1} of {len(dt_columns)}")               
            chunk[col] = pd.to_datetime(chunk[col], errors="coerce")
        
        if "GSN" in chunk.columns:
            chunk["GSN"] = chunk["GSN"].fillna("").astype(str)

        # Save each chunk as a separate Parquet file
        chunk.to_parquet(f"parquet/{base_fn}_{i}.parquet", engine="pyarrow", index=False)

Processing PRESCRIPTIONS.csv.gz / 1 of 32


Processing Chunks:   0%|          | 0/1.0 [00:00<?, ?it/s]c:\Users\ricej\anaconda3\envs\aihc\Lib\site-packages\tqdm\std.py:1181: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  for obj in iterable:
Processing Chunks: 100%|██████████| 1/1.0 [00:13<00:00, 13.09s/it]


Processing PROCEDUREEVENTS_MV.csv.gz / 2 of 32


Processing Chunks: 100%|██████████| 1/1.0 [00:01<00:00,  1.08s/it]


Processing PROCEDURES_ICD.csv.gz / 3 of 32


Processing Chunks: 100%|██████████| 1/1.0 [00:00<00:00,  8.93it/s]


Processing README.md / 4 of 32
	 Skipping since not proper type
Processing SERVICES.csv.gz / 5 of 32


Processing Chunks: 100%|██████████| 1/1.0 [00:00<00:00, 10.98it/s]


Processing SHA256SUMS.txt / 6 of 32
	 Skipping since not proper type
Processing TRANSFERS.csv.gz / 7 of 32


Processing Chunks: 100%|██████████| 1/1.0 [00:00<00:00,  1.73it/s]


# Set up DuckDB helpers

In [6]:
# change working directory to where parquet files are
os.chdir("./parquet")

In [7]:
# column look ups for convenience when wanting to see what columns are available in what tables
mimic_columns = dict()
for tbl in mimic_table_row_counts.keys():
    df = pd.read_parquet(f"{tbl}_0.parquet")
    mimic_columns[tbl] = df.columns

del df
# reformat index objects to lists 
mimic_columns = {col: list(mimic_columns[col]) for col in mimic_columns}
# Setup DuckDB helper functions

In [15]:
def table_finder(columns):
    if type(columns) is str:
        columns = [columns]
    for column in columns:
        for table in mimic_columns.keys():
            if column in mimic_columns[table]:
                print(f"{column} found in {table}")
    return None

In [18]:
def table_check():
    def decorator(fnc):
        def wrapper(*args, **kwargs):
            for arg in args:
                if arg not in mimic_columns.keys():
                    raise ValueError(f"'{arg}' is not in the list of MIMIC III tables")
            return fnc(*args, **kwargs)
        return wrapper
    return decorator

In [19]:
# decorator to change anything I am passing a query to be updated with the parquet files
def duckify():
    def decorator(fnc):
        def wrapper(*args, **kwargs):
            #print(*args)
            new_args = []
            for arg in args:
                if type(arg)==str:
                # print(arg)
                    found=False
                    for tbl in mimic_columns.keys():
                        if arg.find(tbl)>-1:
                            #print(f"before {arg=}")
                            arg = arg.replace(tbl, f"'{tbl}*.parquet'")
                            #print(f"after {arg=}")
                            found=True
                    if not found:
                        raise ValueError("Found no table name in the query string")
                new_args.append(arg)
                #print(args)
            return fnc(*new_args, **kwargs)
        return wrapper
    return decorator


In [20]:
@duckify()
def run_query(qry, print_qry=False):
    """
    Run query and return dataframe, or simply print resulting query
    """
    if print_qry:
        print(qry)
        return None
    else:
        return duckdb.query(qry).to_df()

# Assignment

## 1.  Who are the most recorded patients in the database? 
Find the patients that have the most records across all databases but also the highest average rank

In [23]:
@table_check()
def recon_query(tbl):
    qry = f""" 
    select 
    '{tbl.capitalize()}' as Table,
    subject_id,
    count(*) Count_,
    row_number() over (partition by null order by Count_ desc) Record_Rank
    from {tbl}
    group by 2
    """
    return qry

In [24]:
run_query(recon_query("ADMISSIONS")).head(20)

,Table,SUBJECT_ID,Count_,Record_Rank
0,Admissions,13033,42,1
1,Admissions,11861,34,2
2,Admissions,109,34,3
3,Admissions,5060,31,4
4,Admissions,20643,24,5
5,Admissions,19213,23,6
6,Admissions,7809,22,7
7,Admissions,5727,21,8
8,Admissions,23657,20,9
9,Admissions,11318,19,10


In [ ]:
# build the recon query for every table in MIMIC iii and get the statistics
qry = "union all".join([recon_query(table) for table, columns in mimic_columns.items() if 'SUBJECT_ID' in columns])
recon_df = run_query(f"select subject_id, avg(record_rank), min(record_rank), max(record_rank), sum(Count_) from ({qry}) src group by 1 order by 2")


In [27]:
recon_df.head(15)

,SUBJECT_ID,avg(record_rank),min(record_rank),max(record_rank),sum(Count_)
0,7666,208.000000,21,1963,153790.0
1,27800,334.631579,14,2877,98502.0
2,15046,399.050000,36,2046,85878.0
3,7809,520.000000,4,2245,61514.0
4,55337,523.052632,12,3542,131359.0
5,73713,528.473684,6,8286,86271.0
6,13599,529.000000,22,6028,111196.0
7,46251,563.315789,71,4389,51246.0
8,24900,571.944444,115,1318,88979.0
9,19213,638.888889,5,3953,74122.0


In [34]:
subjects = recon_df.iloc[:3]["SUBJECT_ID"].values
subject_qry_clause = f"subject_id in ({','.join([str(s) for s in subjects])})"

In [35]:
subject_qry_clause

'subject_id in (7666,27800,15046)'

## 2.  What are diseases that they have in common

Write intersection query for diseases 

In [ ]:
table_finder("SUBJECT_ID")

SUBJECT_ID found in ADMISSIONS
SUBJECT_ID found in CALLOUT
SUBJECT_ID found in CHARTEVENTS
SUBJECT_ID found in CPTEVENTS
SUBJECT_ID found in DATETIMEEVENTS
SUBJECT_ID found in DIAGNOSES_ICD
SUBJECT_ID found in DRGCODES
SUBJECT_ID found in ICUSTAYS
SUBJECT_ID found in INPUTEVENTS_CV
SUBJECT_ID found in INPUTEVENTS_MV
SUBJECT_ID found in LABEVENTS
SUBJECT_ID found in MICROBIOLOGYEVENTS
SUBJECT_ID found in NOTEEVENTS
SUBJECT_ID found in OUTPUTEVENTS
SUBJECT_ID found in PATIENTS
SUBJECT_ID found in PRESCRIPTIONS
SUBJECT_ID found in PROCEDUREEVENTS_MV
SUBJECT_ID found in PROCEDURES_ICD
SUBJECT_ID found in SERVICES
SUBJECT_ID found in TRANSFERS


In [139]:
common_diseases_qry = \
"""
select 
diag.*
from DIAGNOSES_ICD subj_diag
left join D_ICD_DIAGNOSES diag
on subj_diag.ICD9_CODE = diag.ICD9_CODE

where subj_diag.SUBJECT_ID = 7666
intersect
select 
diag.*
from DIAGNOSES_ICD subj_diag
left join D_ICD_DIAGNOSES diag
on subj_diag.ICD9_CODE = diag.ICD9_CODE

where subj_diag.SUBJECT_ID = 27800
intersect
select 
diag.*
from DIAGNOSES_ICD subj_diag
left join D_ICD_DIAGNOSES diag
on subj_diag.ICD9_CODE = diag.ICD9_CODE

where subj_diag.SUBJECT_ID = 15046
order by 1
"""

common_diseases_df = run_query(common_diseases_qry)

In [140]:
common_diseases_df

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,660,0389,Septicemia NOS,Unspecified septicemia
1,3147,2859,Anemia NOS,"Anemia, unspecified"
2,4304,4019,Hypertension NOS,Unspecified essential hypertension
3,5279,51881,Acute respiratry failure,Acute respiratory failure
4,5908,5849,Acute kidney failure NOS,"Acute kidney failure, unspecified"


## 3. What medicines do they have in common?

Write intersection query for medicines

In [142]:
run_query("select * from PRESCRIPTIONS limit 10")

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,2214776,6,107064,NaN,2175-06-11,2175-06-12,MAIN,Tacrolimus,Tacrolimus,Tacrolimus,TACR1,021796,469061711.0,1mg Capsule,2.0,mg,2.0,CAP,PO
1,2214775,6,107064,NaN,2175-06-11,2175-06-12,MAIN,Warfarin,Warfarin,Warfarin,WARF5,006562,56017275.0,5mg Tablet,5.0,mg,1.0,TAB,PO
2,2215524,6,107064,NaN,2175-06-11,2175-06-12,MAIN,Heparin Sodium,None,None,HEPAPREMIX,006522,338055002.0,"25,000 unit Premix Bag",NaN,UNIT,1.0,BAG,IV
3,2216265,6,107064,NaN,2175-06-11,2175-06-12,BASE,D5W,None,None,HEPBASE,,0.0,HEPARIN BASE,250.0,ml,250.0,ml,IV
4,2214773,6,107064,NaN,2175-06-11,2175-06-12,MAIN,Furosemide,Furosemide,Furosemide,FURO20,008208,54829725.0,20mg Tablet,20.0,mg,1.0,TAB,PO
5,2214774,6,107064,NaN,2175-06-11,2175-06-15,MAIN,Warfarin,Warfarin,Warfarin,WARF0,014198,56016975.0,Check with MD for Dose,1.0,dose,1.0,dose,PO
6,2215525,6,107064,NaN,2175-06-12,2175-06-12,MAIN,Heparin Sodium,None,None,HEPAPREMIX,006522,338055002.0,"25,000 unit Premix Bag",NaN,UNIT,1.0,BAG,IV
7,2216266,6,107064,NaN,2175-06-12,2175-06-12,BASE,D5W,None,None,HEPBASE,,0.0,HEPARIN BASE,250.0,ml,250.0,ml,IV
8,2215526,6,107064,NaN,2175-06-12,2175-06-13,MAIN,Heparin Sodium,None,None,HEPAPREMIX,006522,338055002.0,"25,000 unit Premix Bag",NaN,UNIT,1.0,BAG,IV
9,2214778,6,107064,NaN,2175-06-12,2175-06-13,MAIN,Warfarin,Warfarin,Warfarin,WARF2,006561,56017075.0,2mg Tab,2.0,mg,1.0,TAB,PO


In [146]:
common_rx_qry = \
"""
select 
drug
from PRESCRIPTIONS 
where SUBJECT_ID = 7666
intersect
select 
drug
from PRESCRIPTIONS 
where SUBJECT_ID =  27800
intersect
select 
drug
from PRESCRIPTIONS 
where SUBJECT_ID =  15046
"""
common_rx_df = run_query(common_rx_qry)

In [ ]:
common_rx_df 

,DRUG
0,Sodium Chloride 0.9% Flush
1,NS
2,Insulin
3,Senna
4,D5W
5,Furosemide
6,Aspirin
7,Midazolam
8,Docusate Sodium
9,Heparin Flush (10 units/ml)


## 4.  What types of ICU initial admissions have they had?

Have subject ID for columns and number of ICU stays by ICU type as columns

In [53]:
run_query("""select * from TRANSFERS limit 10""")

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,EVENTTYPE,PREV_CAREUNIT,CURR_CAREUNIT,PREV_WARDID,CURR_WARDID,INTIME,OUTTIME,LOS
0,657,111,192123,254245.0,carevue,transfer,CCU,MICU,7.0,23.0,2142-04-29 15:27:11,2142-05-04 20:38:33,125.19
1,658,111,192123,NaN,carevue,transfer,MICU,None,23.0,45.0,2142-05-04 20:38:33,2142-05-05 11:46:32,15.13
2,659,111,192123,NaN,carevue,discharge,None,None,45.0,NaN,2142-05-05 11:46:32,NaT,NaN
3,660,111,155897,249202.0,metavision,admit,None,MICU,NaN,52.0,2144-07-01 04:13:59,2144-07-01 05:19:39,1.09
4,661,111,155897,NaN,metavision,transfer,MICU,None,52.0,32.0,2144-07-01 05:19:39,2144-07-01 06:28:29,1.15
5,662,111,155897,249202.0,metavision,transfer,None,MICU,32.0,52.0,2144-07-01 06:28:29,2144-07-01 08:07:16,1.65
6,663,111,155897,NaN,metavision,transfer,MICU,None,52.0,32.0,2144-07-01 08:07:16,2144-07-01 08:13:51,0.11
7,664,111,155897,249202.0,metavision,transfer,None,MICU,32.0,23.0,2144-07-01 08:13:51,2144-07-01 17:56:31,9.71
8,665,111,155897,NaN,metavision,discharge,MICU,None,23.0,NaN,2144-07-01 17:56:31,NaT,NaN
9,666,112,174105,289222.0,carevue,admit,None,MICU,NaN,12.0,2194-06-13 18:41:27,2194-06-14 14:51:17,20.16


In [126]:
icu_profile_df = run_query("""
          select 
          coalesce(CURR_CAREUNIT, cast(CURR_WARDID as varchar(3))) as Ward,
          sum(case when subject_id = 7666 then 1 else 0 end) as Subject_7666_Stays_in_Ward,
          sum(case when subject_id = 27800 then 1 else 0 end) as Subject_27800_Stays_in_Ward,
          sum(case when subject_id = 15046 then 1 else 0 end) as Subject_15046_Stays_in_Ward,
          count(distinct Case when subject_id = 7666 then hadm_id end) as Subject_7666_Stays_by_Admission,
          count(distinct Case when subject_id = 27800 then hadm_id end) as Subject_27800_Stays_by_Admission,
          count(distinct Case when subject_id = 15046 then hadm_id end) as Subject_15046_Stays_by_Admission,
          subject_7666_stays_in_ward + subject_27800_stays_in_ward + subject_15046_stays_in_ward as Total_Stays_In_Ward,
          subject_7666_stays_by_admission + subject_27800_stays_by_admission + subject_15046_stays_by_admission as Total_Admissions_In_Ward
          from TRANSFERS 
          where WARD is not null
          group by 1 
          order by case when regexp_matches(WARD, '^[A-Z]+$') then 1 else 0 end desc, Total_Stays_In_Ward desc
          limit 20
          """)

In [127]:
icu_profile_df

,Ward,Subject_7666_Stays_in_Ward,Subject_27800_Stays_in_Ward,Subject_15046_Stays_in_Ward,Subject_7666_Stays_by_Admission,Subject_27800_Stays_by_Admission,Subject_15046_Stays_by_Admission,Total_Stays_In_Ward,Total_Admissions_In_Ward
0,MICU,7.0,16.0,12.0,6,13,9,35.0,28
1,CCU,1.0,1.0,0.0,1,1,0,2.0,2
2,CSRU,1.0,0.0,0.0,1,0,0,1.0,1
3,SICU,0.0,0.0,0.0,0,0,0,0.0,0
4,NWARD,0.0,0.0,0.0,0,0,0,0.0,0
5,TSICU,0.0,0.0,0.0,0,0,0,0.0,0
6,NICU,0.0,0.0,0.0,0,0,0,0.0,0
7,18.0,0.0,5.0,3.0,0,3,3,8.0,6
8,45.0,5.0,1.0,0.0,4,1,0,6.0,5
9,55.0,1.0,5.0,0.0,1,4,0,6.0,5


## 5.  In their stays in ICU, what kind of chart events have they had?

In [ ]:
run_query("""
          select
          *
          from CHARTEVENTS
          where subject_id = 7666
          limit 10
          """)

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED
0,646224,7666,119064,239385.0,225677,2159-05-26 10:14:00,2159-05-26 12:07:00,20889.0,2.9,2.9,mg/dL,0,0,NaN,NaN
1,646225,7666,119064,239385.0,227073,2159-05-26 10:14:00,2159-05-26 12:07:00,20889.0,16.0,16.0,mEq/L,0,0,NaN,NaN
2,646226,7666,119064,239385.0,227442,2159-05-26 10:14:00,2159-05-26 12:07:00,20889.0,4.5,4.5,mEq/L,0,0,NaN,NaN
3,646227,7666,119064,239385.0,227443,2159-05-26 10:14:00,2159-05-26 12:07:00,20889.0,19.0,19.0,mEq/L,1,0,NaN,NaN
4,646228,7666,119064,239385.0,227456,2159-05-26 10:14:00,2159-05-26 20:46:00,20889.0,3.4,3.4,g/dL,0,0,NaN,NaN
5,646229,7666,119064,239385.0,227457,2159-05-26 10:14:00,2159-05-26 11:15:00,20889.0,68.0,68.0,K/uL,1,0,NaN,NaN
6,646230,7666,119064,239385.0,227465,2159-05-26 10:14:00,2159-05-26 12:07:00,20889.0,20.8,20.8,sec,1,0,NaN,NaN
7,646231,7666,119064,239385.0,227466,2159-05-26 10:14:00,2159-05-26 12:07:00,20889.0,34.4,34.4,sec,0,0,NaN,NaN
8,646232,7666,119064,239385.0,227467,2159-05-26 10:14:00,2159-05-26 12:07:00,20889.0,2.0,2.0,None,1,0,NaN,NaN
9,646233,7666,119064,239385.0,225668,2159-05-26 11:57:00,2159-05-26 11:58:00,20889.0,1.8,1.8,mmol/L,0,0,NaN,NaN


array([ 7666, 27800, 15046], dtype=int64)

In [128]:
chart_events_df = run_query(f"""
          select 
          --cv.ITEMID,
          items.LABEL,
          count(distinct cv.itemid) as Item_Code_Count,
          count(distinct case when subject_id = 7666 then charttime else null end) Chart_Event_7666,  
          count(distinct case when subject_id = 27800 then charttime else null end) Chart_Event_27800, 
          count(distinct case when subject_id = 15046 then charttime else null end) Chart_Event_15046
          from CHARTEVENTS cv
          inner join D_ITEMS items
           on cv.ITEMID = items.ITEMID
          where {subject_qry_clause} 
          and cv.value is not null
          group by 1
          having least(Chart_Event_7666, Chart_Event_27800, Chart_Event_15046) > 0
          order by chart_event_7666 + chart_event_27800 + chart_event_15046 desc
          limit 20
          """)

In [129]:
chart_events_df

,LABEL,Item_Code_Count,Chart_Event_7666,Chart_Event_27800,Chart_Event_15046
0,Heart Rate,2,2146,1838,1243
1,Respiratory Rate,2,2131,1838,1243
2,Mean Airway Pressure,2,207,323,225
3,Peak Insp. Pressure,2,123,305,165
4,Arterial Base Excess,2,364,148,58
5,BUN,2,276,92,72
6,Creatinine,2,276,92,72
7,Minute Volume,2,1,336,88
8,Magnesium,2,249,80,66
9,Phosphorous,2,251,77,64


## 6.  Let's compare their average most common vitals by visit

In [110]:
mimic_columns["CHARTEVENTS"]

['ROW_ID',
 'SUBJECT_ID',
 'HADM_ID',
 'ICUSTAY_ID',
 'ITEMID',
 'CHARTTIME',
 'STORETIME',
 'CGID',
 'VALUE',
 'VALUENUM',
 'VALUEUOM',
 'WARNING',
 'ERROR',
 'RESULTSTATUS',
 'STOPPED']

In [121]:
run_query(
f"""
select 
coalesce(s7666.admission_number, s27800.admission_number, s15046.admission_number) as Admission_Number_for_Patient,
s7666.Avg_Heart_Rate as Avg_Heart_Rate_7666,
s27800.Avg_Heart_Rate as Avg_Heart_Rate_27800,
s15046.Avg_Heart_Rate as Avg_Heart_Rate_15046,
s7666.Avg_Respiratory_Rate as Avg_Respiratory_Rate_7666,
s27800.Avg_Respiratory_Rate as Avg_Respiratory_Rate_27800,
s15046.Avg_Respiratory_Rate as Avg_Respiratory_Rate_15046,
s7666.Avg_Mean_Airway_Pressure as Avg_Mean_Airway_Pressure_7666,
s27800.Avg_Mean_Airway_Pressure as Avg_Mean_Airway_Pressure_27800,
s15046.Avg_Mean_Airway_Pressure as Avg_Mean_Airway_Pressure_15046,
s7666.Avg_Peak_Insp_Pressure as Avg_Peak_Insp_Pressure_7666,
s27800.Avg_Peak_Insp_Pressure as Avg_Peak_Insp_Pressure_27800,
s15046.Avg_Peak_Insp_Pressure as Avg_Peak_Insp_Pressure_15046,
s7666.Avg_Arterial_Base_Excess as Avg_Arterial_Base_Excess_7666,
s27800.Avg_Arterial_Base_Excess as Avg_Arterial_Base_Excess_27800,
s15046.Avg_Arterial_Base_Excess as Avg_Arterial_Base_Excess_15046
from (
  select 
  subject_id,
  hadm_id,
  min(charttime) as First_Chart_Event,
  avg(case when label = 'Heart Rate' then valuenum else null end) as Avg_Heart_Rate,
  avg(case when label = 'Respiratory Rate' then valuenum else null end) as Avg_Respiratory_Rate,
  avg(case when label = 'Mean Airway Pressure' then valuenum else null end) as Avg_Mean_Airway_Pressure,
  avg(case when label = 'Peak Insp. Pressure' then valuenum else null end) as Avg_Peak_Insp_Pressure,
  avg(case when label = 'Arterial Base Excess' then valuenum else null end) as Avg_Arterial_Base_Excess,
  Row_Number() over (partition by subject_id order by min(charttime)) as Admission_Number
  from CHARTEVENTS cv
  inner join D_ITEMS items
    on cv.ITEMID = items.ITEMID
  where items.LABEL in ('Heart Rate', 'Respiratory Rate', 'Mean Airway Pressure', 'Peak Insp. Pressure', 'Arterial Base Excess')
  and subject_id = 7666
  group by 1,2
) s7666
full outer join (
  select 
  subject_id,
  hadm_id,
  min(charttime) as First_Chart_Event,
  avg(case when label = 'Heart Rate' then valuenum else null end) as Avg_Heart_Rate,
  avg(case when label = 'Respiratory Rate' then valuenum else null end) as Avg_Respiratory_Rate,
  avg(case when label = 'Mean Airway Pressure' then valuenum else null end) as Avg_Mean_Airway_Pressure,
  avg(case when label = 'Peak Insp. Pressure' then valuenum else null end) as Avg_Peak_Insp_Pressure,
  avg(case when label = 'Arterial Base Excess' then valuenum else null end) as Avg_Arterial_Base_Excess,
  Row_Number() over (partition by subject_id order by min(charttime)) as Admission_Number
  from CHARTEVENTS cv
  inner join D_ITEMS items
    on cv.ITEMID = items.ITEMID
  where items.LABEL in ('Heart Rate', 'Respiratory Rate', 'Mean Airway Pressure', 'Peak Insp. Pressure', 'Arterial Base Excess')
  and subject_id = 27800
  group by 1,2
) s27800
 on s7666.admission_number = s27800.admission_number
full outer join (
  select 
  subject_id,
  hadm_id,
  min(charttime) as First_Chart_Event,
  avg(case when label = 'Heart Rate' then valuenum else null end) as Avg_Heart_Rate,
  avg(case when label = 'Respiratory Rate' then valuenum else null end) as Avg_Respiratory_Rate,
  avg(case when label = 'Mean Airway Pressure' then valuenum else null end) as Avg_Mean_Airway_Pressure,
  avg(case when label = 'Peak Insp. Pressure' then valuenum else null end) as Avg_Peak_Insp_Pressure,
  avg(case when label = 'Arterial Base Excess' then valuenum else null end) as Avg_Arterial_Base_Excess,
  Row_Number() over (partition by subject_id order by min(charttime)) as Admission_Number
  from CHARTEVENTS cv
  inner join D_ITEMS items
    on cv.ITEMID = items.ITEMID
  where items.LABEL in ('Heart Rate', 'Respiratory Rate', 'Mean Airway Pressure', 'Peak Insp. Pressure', 'Arterial Base Excess')
  and subject_id = 15046
  group by 1,2
) s15046
 on s27800.admission_number = s15046.admission_number
order by 1
""")

,Admission_Number_for_Patient,Avg_Heart_Rate_7666,Avg_Heart_Rate_27800,Avg_Heart_Rate_15046,Avg_Respiratory_Rate_7666,Avg_Respiratory_Rate_27800,Avg_Respiratory_Rate_15046,Avg_Mean_Airway_Pressure_7666,Avg_Mean_Airway_Pressure_27800,Avg_Mean_Airway_Pressure_15046,Avg_Peak_Insp_Pressure_7666,Avg_Peak_Insp_Pressure_27800,Avg_Peak_Insp_Pressure_15046,Avg_Arterial_Base_Excess_7666,Avg_Arterial_Base_Excess_27800,Avg_Arterial_Base_Excess_15046
0,1,83.000000,69.770115,64.293907,19.324324,22.988506,17.946237,NaN,8.363636,10.081967,NaN,24.125000,24.960000,1.000000,10.625000,0.277778
1,2,82.826829,72.250000,67.174699,21.645161,28.178571,18.515060,13.246154,NaN,10.025714,26.037500,NaN,21.391373,1.047619,NaN,-0.862069
2,3,78.384615,78.208333,77.544304,22.269231,32.166667,21.772152,NaN,7.500000,9.500000,NaN,NaN,23.166667,NaN,12.000000,-1.666667
3,4,80.714286,69.700000,78.250000,17.888889,27.700000,17.972973,16.337500,NaN,NaN,28.537500,NaN,NaN,-1.000000,3.000000,1.000000
4,5,83.652174,60.769231,83.931818,20.028986,18.381526,20.022727,8.500000,9.769231,NaN,22.500000,26.849057,NaN,-0.583333,1.913043,NaN
5,6,73.969231,61.751701,56.627907,19.830769,21.387755,16.883721,NaN,12.807692,8.307692,NaN,31.413043,18.166667,NaN,6.478261,-0.500000
6,7,74.493631,75.333333,72.017544,24.585987,29.666667,18.175439,12.583333,NaN,NaN,18.250000,NaN,NaN,-4.269231,7.000000,NaN
7,8,83.722266,57.771552,64.337979,22.941640,21.508621,17.325175,11.310811,10.018519,6.435484,20.886598,22.265306,12.542373,2.208633,8.272727,3.400000
8,9,NaN,72.984456,NaN,NaN,26.984456,NaN,NaN,8.777778,NaN,NaN,25.000000,NaN,NaN,6.285714,NaN
9,10,NaN,60.750000,NaN,NaN,19.861386,NaN,NaN,9.432432,NaN,NaN,23.583333,NaN,NaN,11.666667,NaN


In [131]:
run_query(
f"""
select 
subject_id,
count(distinct hadm_id) as Admissions
from CHARTEVENTS
where {subject_qry_clause}
group by 1
order by 1
"""
)

,SUBJECT_ID,Admissions
0,7666,8
1,15046,8
2,27800,13


## 7. How many other people have the same diseases? 

In [134]:
common_diseases_df["ICD9_CODE"].values

array(['0389', '5849', '2859', '51881', '4019'], dtype=object)

In [153]:
print(common_diseases_qry)


select 
diag.*
from DIAGNOSES_ICD subj_diag
left join D_ICD_DIAGNOSES diag
on subj_diag.ICD9_CODE = diag.ICD9_CODE

where subj_diag.SUBJECT_ID = 7666
intersect
select 
diag.*
from DIAGNOSES_ICD subj_diag
left join D_ICD_DIAGNOSES diag
on subj_diag.ICD9_CODE = diag.ICD9_CODE

where subj_diag.SUBJECT_ID = 27800
intersect
select 
diag.*
from DIAGNOSES_ICD subj_diag
left join D_ICD_DIAGNOSES diag
on subj_diag.ICD9_CODE = diag.ICD9_CODE

where subj_diag.SUBJECT_ID = 15046
order by 1



In [161]:
mimic_columns["DIAGNOSES_ICD"]

['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'ICD9_CODE']

In [178]:
run_query(
f"""
select 
subj_diag.ICD9_CODE,
diag.SHORT_TITLE,
diag.LONG_TITLE,
count(distinct subj_diag.subject_id) as Patients_Affected
from DIAGNOSES_ICD subj_diag
left join D_ICD_DIAGNOSES diag
 on subj_diag.ICD9_CODE = diag.ICD9_CODE
where subj_diag.ICD9_CODE in (
select 
    ICD9_CODE 
    from (
        {common_diseases_qry}
    ) 
)
group by 1,2,3
order by 2
""")

,ICD9_CODE,SHORT_TITLE,LONG_TITLE,Patients_Affected
0,5849,Acute kidney failure NOS,"Acute kidney failure, unspecified",7687
1,51881,Acute respiratry failure,Acute respiratory failure,6719
2,2859,Anemia NOS,"Anemia, unspecified",4993
3,4019,Hypertension NOS,Unspecified essential hypertension,17613
4,0389,Septicemia NOS,Unspecified septicemia,3433


## 8. How many other people use the same meds? 

In [166]:
print(common_rx_qry)


select 
drug
from PRESCRIPTIONS 
where SUBJECT_ID = 7666
intersect
select 
drug
from PRESCRIPTIONS 
where SUBJECT_ID =  27800
intersect
select 
drug
from PRESCRIPTIONS 
where SUBJECT_ID =  15046



In [177]:
run_query(
f"""
select 
drug,
count(distinct subject_id) as Patients_Administered_Rx
from PRESCRIPTIONS
where drug in (
select 
    DRUG 
    from (
        {common_rx_qry}
    ) 
)
group by 1
order by 2 desc
""")

,DRUG,Patients_Administered_Rx
0,Potassium Chloride,29711
1,Sodium Chloride 0.9% Flush,29387
2,Acetaminophen,28821
3,Magnesium Sulfate,26120
4,Insulin,25235
5,Heparin,24444
6,Docusate Sodium,23669
7,D5W,21279
8,Iso-Osmotic Dextrose,21098
9,Morphine Sulfate,20825


## 9. How many people have the same diseases AND meds? 

In [ ]:
run_query(
f"""
select
count(distinct rx.subject_id)
from (
    select 
    distinct
    subject_id,
    drug
    from PRESCRIPTIONS
) rx
inner join (
    select 
    distinct
    subject_id,
    icd9_code
    from DIAGNOSES_ICD
) diag
on rx.subject_id = diag.subject_id
where drug in (
select 
    DRUG 
    from (
        {common_rx_qry}
    ) 
)
and icd9_code in (
select 
    ICD9_CODE 
    from (
        {common_diseases_qry}
    ) 
)
and rx.subject_id not in (7666, 27800, 15046)
"""
)

,count(DISTINCT rx.subject_id)
0,24586


## 10.  What are the rarest diagnoses and meds?

In [173]:
run_query(
f"""
select 
subj_diag.icd9_code,
diag.short_title,
diag.long_title,
count(distinct subj_diag.subject_id) as Patients_Affected
from DIAGNOSES_ICD subj_diag
inner join D_ICD_DIAGNOSES diag
 on subj_diag.ICD9_CODE = diag.ICD9_CODE
--where subj_diag.subject_id in (7666, 27800, 15046)
group by 1,2,3
order by 4 
limit 100
"""
)

,ICD9_CODE,SHORT_TITLE,LONG_TITLE,Patients_Affected
0,E975,Legal intervention NEC,Injury due to legal intervention by other spec...,1
1,4481,"Nevus, non-neoplastic","Nevus, non-neoplastic",1
2,3029,Psychosexual dis NOS,Unspecified psychosexual disorder,1
3,92701,Crush inj scapul region,Crushing injury of scapular region,1
4,62981,Rec preg loss wo cur prg,Recurrent pregnancy loss without current pregn...,1
...,...,...,...,...
95,75613,Cong absence of vertebra,"Absence of vertebra, congenital",1
96,36002,Panophthalmitis,Panophthalmitis,1
97,4957,"""ventilation"" pneumonit","""Ventilation"" pneumonitis",1
98,V065,Vaccination for Td-DT,Need for prophylactic vaccination and inoculat...,1


In [174]:
run_query(
f"""
select 
drug,
count(distinct subject_id) as Patients_Given_Rx
from PRESCRIPTIONS
group by 1
order by 2 
limit 100
"""
)

,DRUG,Patients_Given_Rx
0,Valproic,1
1,Aluminum,1
2,Theo Liquid,1
3,eucerin,1
4,racemic epinephrine,1
...,...,...
95,Dexamethasone Sod Phos Ophth,1
96,famotidine,1
97,Miconazole,1
98,Nicotinic Acid SR,1


In [188]:
run_query(
"""
select drug from PRESCRIPTIONS where upper(drug) like '%VICODIN%'
"""
)

,DRUG
0,Vicodin
1,VICOdin


## 11.  Bonus:  What are the most frequent diagnoses and meds combinations?  

In [ ]:
run_query(
"""
select 
diag.hadm_id,
diag.icd9_code,
diag_text.short_title,
diag_text.long_title,
rx.drug
from DIAGNOSES_ICD diag
inner join D_ICD_DIAGNOSES diag_text
    on diag.icd9_code = diag_text.icd9_code
inner join PRESCRIPTIONS rx
    on diag.hadm_id = rx.hadm_id
where diag.subject_id = 7666
"""
)


,HADM_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE,DRUG
0,178499,V180,Fam hx-diabetes mellitus,Family history of diabetes mellitus,Sirolimus
1,178499,V180,Fam hx-diabetes mellitus,Family history of diabetes mellitus,Pravastatin
2,178499,V180,Fam hx-diabetes mellitus,Family history of diabetes mellitus,Sulfameth/Trimethoprim SS
3,178499,V180,Fam hx-diabetes mellitus,Family history of diabetes mellitus,Dextrose 50%
4,178499,V180,Fam hx-diabetes mellitus,Family history of diabetes mellitus,Docusate Sodium
...,...,...,...,...,...
16544,119064,25040,DMII renl nt st uncntrld,"Diabetes with renal manifestations, type II or...",Acetaminophen
16545,119064,25040,DMII renl nt st uncntrld,"Diabetes with renal manifestations, type II or...",Ferrous Sulfate
16546,119064,25040,DMII renl nt st uncntrld,"Diabetes with renal manifestations, type II or...",Mycophenolate Mofetil
16547,119064,25040,DMII renl nt st uncntrld,"Diabetes with renal manifestations, type II or...",LaMIVudine


In [192]:
common_combo_df = run_query(
"""
select 
diag.icd9_code,
diag_text.short_title,
diag_text.long_title,
rx.drug,
count(distinct diag.hadm_id) as Admissions_with_Combo,
count(distinct diag.subject_id) as Patients_with_Combo
from DIAGNOSES_ICD diag
inner join D_ICD_DIAGNOSES diag_text
    on diag.icd9_code = diag_text.icd9_code
inner join PRESCRIPTIONS rx
    on diag.hadm_id = rx.hadm_id
group by 1,2,3,4
order by 5 desc
"""
)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [193]:
common_combo_df.head(20)

,ICD9_CODE,SHORT_TITLE,LONG_TITLE,DRUG,Admissions_with_Combo,Patients_with_Combo
0,4019,Hypertension NOS,Unspecified essential hypertension,Sodium Chloride 0.9% Flush,15941,13798
1,4019,Hypertension NOS,Unspecified essential hypertension,Acetaminophen,15020,13370
2,4019,Hypertension NOS,Unspecified essential hypertension,Potassium Chloride,14927,13232
3,4019,Hypertension NOS,Unspecified essential hypertension,Magnesium Sulfate,13838,12320
4,4019,Hypertension NOS,Unspecified essential hypertension,Insulin,13632,12036
5,4019,Hypertension NOS,Unspecified essential hypertension,Docusate Sodium,12427,11142
6,4019,Hypertension NOS,Unspecified essential hypertension,Heparin,12033,10433
7,4019,Hypertension NOS,Unspecified essential hypertension,Furosemide,10814,9747
8,4019,Hypertension NOS,Unspecified essential hypertension,D5W,10780,9841
9,4019,Hypertension NOS,Unspecified essential hypertension,Morphine Sulfate,10270,9589
